In [1]:
import easydict
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import os
import time
import tensorflow as tf
from PIL import ImageFont, ImageDraw, Image

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [3]:
model = load_model("CNN_best_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
# TF-Pose, HOG descriptor 객체 생성 
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

[2021-01-29 02:27:46,955] [TfPoseEstimator] [INFO] loading graph from C:\Users\pc\Desktop\2021_UGRP\personal\Nwoo\Gesture_Recognition\main_work\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [8]:
count = 0
cap = cv2.VideoCapture(1)
categories = ['pedestrian', 'sitter', 'taxier']

while True:
    ret, image = cap.read()
    people_bbox = dict()

    if not ret:
        break
    # Bounding Box 그리기
    detected, _ = hog.detectMultiScale(image)
    
    # Skeleton 그리기 with Background
    humans = e.inference(image, upsample_size=4.0)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    
    for (x,y,w,h) in detected:    
        # 가로 200pixel 세로 300pixel 이하 제외
#         if w < 100 and h < 200: continue

        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
        
        # image resizing and reshape (Preprocessing)
        input_image = image[int(y):int(y+h), int(x):int(x+w)]
        img = Image.fromarray(input_image, "RGB")
#         img = Image.open(input_image)
        img = img.convert("RGB")
        img = img.resize((500,500))
        data = np.asarray(img)
        
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 500,500,3)
        
        # model prediction
        result = [np.argmax(value) for value in model.predict(X)] 
        result = categories[result[0]]
        people_bbox[(x,y,w,h)] = result
        
    
    # esc 누르면 종료
    if cv2.waitKey(10) == 27:                    
        break
    print('%d.jpg done' % count)
    count += 1
    
    # Class name print 
    for (x,y,w,h) in people_bbox:
        gesture = people_bbox[(x,y,w,h)]
        cv2.putText(image, gesture, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)
    
    cv2.imshow("Gesture_Recognition", image)
#윈도우 종료
cap.release()
cv2.destroyWindow('Gesture_Recognition')

0.jpg done
1.jpg done
2.jpg done
3.jpg done
4.jpg done
5.jpg done
6.jpg done
7.jpg done
8.jpg done
9.jpg done
10.jpg done
11.jpg done
12.jpg done
13.jpg done
14.jpg done
15.jpg done
16.jpg done
17.jpg done
18.jpg done
19.jpg done
20.jpg done
21.jpg done
22.jpg done
23.jpg done
24.jpg done
25.jpg done
26.jpg done
27.jpg done
28.jpg done
29.jpg done
30.jpg done
31.jpg done
32.jpg done
33.jpg done
34.jpg done
35.jpg done
36.jpg done
37.jpg done
38.jpg done
39.jpg done
40.jpg done
41.jpg done
42.jpg done
43.jpg done
44.jpg done
45.jpg done
46.jpg done
47.jpg done
48.jpg done
49.jpg done
50.jpg done
51.jpg done
52.jpg done
53.jpg done
54.jpg done
55.jpg done
56.jpg done
57.jpg done
58.jpg done
59.jpg done


# Test

In [14]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

people_bbox = dict()
categories = ['pedestrian', 'sitter', 'taxier']
image = cv2.imread('Test.jpg', cv2.IMREAD_COLOR)

# Bounding Box 그리기
detected, _ = hog.detectMultiScale(image)

# Skeleton 그리기 with Background
humans = e.inference(image, upsample_size=4.0)
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)    

for (x, y, w, h) in detected:
    if w < 200 and h < 300: continue
    cv2.rectangle(image, (x,y),(x+w,y+h), (255,0,0), 3) # 파란색
    # image resizing and reshape (Preprocessing)
    input_image = image[int(y):int(y+h), int(x):int(x+w)]
    img = Image.fromarray(input_image, "RGB")
#     img = Image.open(input_image)
    img = img.convert("RGB")
    img = img.resize((500,500))
    data = np.asarray(img)

    X = np.array(data)
    X = X.astype("float") / 256
    X = X.reshape(-1, 500,500,3)

    # model prediction
    pred = model.predict(X)
    result = [np.argmax(value) for value in pred] 
    result = categories[result[0]]
    people_bbox[(x,y,w,h)] = result
          
    
# Class name print 
for (x,y,w,h) in people_bbox:
    text = people_bbox[(x,y,w,h)]
    cv2.putText(image, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)

    
cv2.imshow("Gesture_Recognition", image)
cv2.imwrite("Test_final.jpg", image)

cv2.waitKey(0)
cv2.destroyAllWindows()

# #윈도우 종료
# cap.release()
# cv2.destroyWindow('Gesture_Recognition')

[2021-01-29 00:12:50,285] [TfPoseEstimator] [INFO] loading graph from C:\Users\pc\Desktop\2021_UGRP\personal\Nwoo\Gesture_Recognition\main_work\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)
